<a href="https://colab.research.google.com/github/food-hazard-detection-semeval-2025/food-hazard-detection-semeval-2025.github.io/blob/main/code/The_Food_Hazard_Detection_Challenge_SemEval_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# incidents_rest.csv is the dataset besides the trial
# incidents_sample.csv is the trial dataset

In [ ]:
import pandas as pd
data = pd.read_csv('incidents_rest.csv', index_col=0)
trial = pd.read_csv('incidents_sample.csv', index_col=0)
trial.sample()

,year,month,day,title,product,product-category,product-title,hazard,hazard-category,hazard-title,language,country
91,2021,3,31,Consumption of Heng Le brand Blanched Bitter A...,apricots,fruits and vegetables,"(38,51)",high content of cyanide,chemical,NaN,en,ca


In [ ]:
from sklearn.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.2, random_state=2024)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

text_clf_lr = Pipeline([
    ('vect', TfidfVectorizer(strip_accents='unicode', analyzer='char', ngram_range=(2,5), max_df=0.5, min_df=5)),
     ('clf', LogisticRegression(max_iter=1000)),
    ])

# Evaluation
* On 20% of the training data
* Aligned (should be) to [our paper](https://openreview.net/pdf?id=SAeSTkdtlC-)

In [ ]:
from sklearn.metrics import classification_report, f1_score
for label in ('hazard-category', 'product-category', 'hazard', 'product'):
  print(label.upper())
  text_clf_lr.fit(trainset.title, trainset[label])
  lr_predictions = text_clf_lr.predict(testset.title)
  print(f'macro: {f1_score(testset[label], lr_predictions, zero_division=0, average="macro"):.2f}')
  print(f'micro: {f1_score(testset[label], lr_predictions, zero_division=0, average="micro"):.2f}')

HAZARD-CATEGORY
macro: 0.48
micro: 0.81
PRODUCT-CATEGORY
macro: 0.38
micro: 0.64
HAZARD
macro: 0.14
micro: 0.58
PRODUCT
macro: 0.07
micro: 0.27


# Evaluation on the trial data
* And holding the predictions (one per text)

In [ ]:
from sklearn.metrics import classification_report, f1_score
predictions = {}
for label in ('hazard-category', 'product-category', 'hazard', 'product'):
  print(label.upper())
  text_clf_lr.fit(trainset.title, trainset[label])
  predictions[label] = text_clf_lr.predict(trial.title)
  print(f"F1-micro: {f1_score(trial[label], predictions[label], average='micro'):.2f}")
  print(f"F1-macro: {f1_score(trial[label], predictions[label], average='macro'):.2f}")

HAZARD-CATEGORY
F1-micro: 0.86
F1-macro: 0.61
PRODUCT-CATEGORY
F1-micro: 0.65
F1-macro: 0.52
HAZARD
F1-micro: 0.61
F1-macro: 0.34
PRODUCT
F1-micro: 0.29
F1-macro: 0.15


In [ ]:
for label in ('hazard-category', 'product-category', 'hazard', 'product'):
  trial['lr-'+label] = predictions[label]

In [ ]:
trial

,year,month,day,title,product,product-category,product-title,hazard,hazard-category,hazard-title,language,country,lr-hazard,lr-product,lr-hazard-category,lr-product-category
0,1998,11,27,Archive - PRODUCT ALERT - POSSIBLE BOTULISM TO...,olives,fruits and vegetables,"(84,89)",clostridium botulinum,biological,NaN,en,ca,listeria monocytogenes,ice cream,biological,"meat, egg and dairy products"
1,2002,7,25,Heinz—Baked Beans in Tomato Sauce,ready to eat - cook meals,prepared dishes and snacks,NaN,packaging defect,packaging defect,"(0,4)",en,au,milk and products thereof,cakes,allergens,"soups, broths, sauces and condiments"
2,2003,9,24,Woolworths Limited—Premium Beef Mince and Regu...,minced beef,"meat, egg and dairy products","(27,30)|(50,53)",metal fragment,foreign bodies,"(0,17)",en,au,metal fragment,beef,foreign bodies,"meat, egg and dairy products"
3,2004,2,10,Azzura Gelati—Azzura Kisses,ice cream,ices and desserts,NaN,other not classified allergen hazards,allergens,"(0,19)",en,au,listeria monocytogenes,ice cream,biological,"meat, egg and dairy products"
4,2005,4,29,Tender Choice Foods(Aust) Pty Ltd—Oven Roasted...,ready to eat - cook meals,prepared dishes and snacks,NaN,listeria monocytogenes,biological,"(0,32)",en,au,listeria monocytogenes,chicken based products,biological,"meat, egg and dairy products"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2021,9,13,Prepackaged frozen lobster sample detected wit...,frozen lobster,seafood,"(12,25)",sulphur dioxide,fraud,NaN,en,hk,sulphur dioxide,ready to eat - cook meals,fraud,fruits and vegetables
96,2021,10,25,Cedar’s Mediterranean Foods Issues Allergy Ale...,organic chilled hummus,prepared dishes and snacks,"(82,88)",other,allergens,"(0,26)|(90,102)",en,us,milk and products thereof,ice cream,allergens,cereals and bakery products
97,2022,5,3,"Safeway Fresh Food, LLC Recalls Ready-To-Eat S...",prepared salads,prepared dishes and snacks,"(45,49)",fish and products thereof,allergens,"(0,22)",en,us,milk and products thereof,chicken based products,allergens,"meat, egg and dairy products"
98,2022,5,23,"Cargill Voluntarily Recalls 795, 8oz Boxes of ...",chocolate products,"cocoa and cocoa preparations, coffee and tea","(46,52)",salmonella,biological,"(0,6)",en,us,salmonella,peanuts,biological,"nuts, nut products and seeds"


In [ ]:
from sklearn.metrics import f1_score

def compute_score(hazards_true, products_true, hazards_pred, products_pred):
  # compute f1 for hazards:
  f1_hazards = f1_score(
    hazards_true,
    hazards_pred,
    average='macro'
  )

  # compute f1 for products:
  f1_products = f1_score(
    products_true[hazards_pred == hazards_true],
    products_pred[hazards_pred == hazards_true],
    average='macro'
  )

  return (f1_hazards + f1_products) / 2.

In [ ]:
print('Score Sub-Task 1:', compute_score(trial['hazard-category'], trial['product-category'], trial['lr-hazard-category'], trial['lr-product-category']))
print('Score Sub-Task 2:', compute_score(trial['hazard'], trial['product'], trial['lr-hazard'], trial['lr-product']))


Score Sub-Task 1: 0.5359963155955866
Score Sub-Task 2: 0.2559326349626896
